# Implementacja modelu uczenia maszynowego na potrzeby predykcji kursu indeksu WIG20

Ideą numer 1 jest próba wyłonienia najbardziej istotnych informacji, które zwiększają prawdopodobieństwo podjęcia właściwej decyzji inwestycyjnej.
Ideą numer 2 jest stworzenie modelu uczenia maszynowego, aby sprawdzić jego przydatność w tradingu. Używana do budowy modeli jest bibliteka scikit-learn.
Idea numer 3 to próba wyłonienia powiązań między wskaźnikami, które z wyprzedzeniem informują o zbliżających się sygnałach inwestycyjnych

### Instalacja niezbędnych bibliotek

Bazuję na danych historycznych pochodzących z serwisu: https://www.dukascopy.com/swiss/pl/marketwatch/historical/
Dukascopy daje możliwość pobierania danych za pomocą specjalnej biblioteki pythonowej, co jest bardzo wygodnym rozwiązaniem. Nazwa biblioteki: dukascopy-python

In [157]:
#pip install dukascopy-python

In [158]:
#pip install --upgrade dukascopy-python

Biblioteka dukascopy-python pobiera dane prosto do struktury danych Pandas DataFrame, więc trzeba zainstalować biblitekę Pandas

In [159]:
#pip install pandas

In [160]:
#pip install --upgrade pandas

Sprawdzenie możliwości biblioteki

Niezbędna będzie malipulacja datami: biblioteka datetime

In [161]:
#pip install datetime

In [162]:
#pip install --upgrade datetime

Skoro ML to biblioteka scikit-learn

In [163]:
#pip install scikit-learn

In [164]:
#pip intall --upgrade scikit-learn

Jakaś biblioteka do analizy technicznej

In [165]:
#pip install pandas-ta

In [166]:
#pip install --ipgrade pandas-ta

### Import bibliotek

In [167]:
from datetime import datetime
#
import dukascopy_python as dcp
#
import pandas as pd
#
import pandas_ta as ta

Wyświetlenie dokumentacji biblioteki dukascopy_python

In [168]:
help(dcp)

Help on package dukascopy_python:

NAME
    dukascopy_python

PACKAGE CONTENTS
    instruments

FUNCTIONS
    fetch(instrument: str, interval: str, offer_side: str, start: datetime.datetime, end: datetime.datetime, max_retries: int = 7, limit: int = 30000, debug=False)

    live_fetch(instrument: str, interval_value: int, time_unit: str, offer_side: str, start: datetime.datetime, end: datetime.datetime, max_retries: int = 7, limit: int = 30000, debug=False)

    sleep(...)
        sleep(seconds)

        Delay execution for a given number of seconds.  The argument may be
        a floating-point number for subsecond precision.

DATA
    INTERVAL_DAY_1 = '1DAY'
    INTERVAL_HOUR_1 = '1HOUR'
    INTERVAL_HOUR_4 = '4HOUR'
    INTERVAL_MIN_1 = '1MIN'
    INTERVAL_MIN_10 = '10MIN'
    INTERVAL_MIN_15 = '15MIN'
    INTERVAL_MIN_30 = '30MIN'
    INTERVAL_MIN_5 = '5MIN'
    INTERVAL_MONTH_1 = '1MONTH'
    INTERVAL_SEC_1 = '1SEC'
    INTERVAL_SEC_10 = '10SEC'
    INTERVAL_SEC_30 = '30SEC'
    I

Sprawdzenie listy dostępnych instrumentów

In [169]:
import dukascopy_python.instruments as dcpi

In [170]:
help(dcpi)

Help on module dukascopy_python.instruments in dukascopy_python:

NAME
    dukascopy_python.instruments

DATA
    INSTRUMENT_AUSTRIA_ANDR_AT_EUR = 'ANDR.AT/EUR'
    INSTRUMENT_AUSTRIA_EBS_AT_EUR = 'EBS.AT/EUR'
    INSTRUMENT_AUSTRIA_POST_AT_EUR = 'POST.AT/EUR'
    INSTRUMENT_AUSTRIA_RBI_AT_EUR = 'RBI.AT/EUR'
    INSTRUMENT_AUSTRIA_TKA_AT_EUR = 'TKA.AT/EUR'
    INSTRUMENT_AUSTRIA_VER_AT_EUR = 'VER.AT/EUR'
    INSTRUMENT_AUSTRIA_VIG_AT_EUR = 'VIG.AT/EUR'
    INSTRUMENT_AUSTRIA_VOE_AT_EUR = 'VOE.AT/EUR'
    INSTRUMENT_AUSTRIA_WIE_AT_EUR = 'WIE.AT/EUR'
    INSTRUMENT_BELGIUM_ABI_BE_EUR = 'ABI.BE/EUR'
    INSTRUMENT_BELGIUM_AGS_BE_EUR = 'AGS.BE/EUR'
    INSTRUMENT_BELGIUM_BELG_BE_EUR = 'BELG.BE/EUR'
    INSTRUMENT_BELGIUM_KBC_BE_EUR = 'KBC.BE/EUR'
    INSTRUMENT_BELGIUM_SOLB_BE_EUR = 'SOLB.BE/EUR'
    INSTRUMENT_BELGIUM_UCB_BE_EUR = 'UCB.BE/EUR'
    INSTRUMENT_BELGIUM_UMI_BE_EUR = 'UMI.BE/EUR'
    INSTRUMENT_BND_CFD_BUND_TR_EUR = 'BUND.TR/EUR'
    INSTRUMENT_BND_CFD_UKGILT_TR_GBP = 'UKGILT.

Sprawdzenie możliwości biblioteki Pandas TA

In [171]:
help(ta)

Help on package pandas_ta:

NAME
    pandas_ta

PACKAGE CONTENTS
    candles (package)
    core
    custom
    cycles (package)
    momentum (package)
    overlap (package)
    performance (package)
    statistics (package)
    trend (package)
    utils (package)
    volatility (package)
    volume (package)

SUBMODULES
    alphavantage
    data
    yahoofinance

FUNCTIONS
    localtime(...)
        localtime([seconds]) -> (tm_year,tm_mon,tm_mday,tm_hour,tm_min,
                                  tm_sec,tm_wday,tm_yday,tm_isdst)

        Convert seconds since the Epoch to a time tuple expressing local time.
        When 'seconds' is not passed in, convert the current time instead.

    mfloor = floor(x, /)
        Return the floor of x as an Integral.

        This is the largest integer <= x.

    mul(a, b, /)
        Same as a * b.

    npArray = array(...)
        array(object, dtype=None, *, copy=True, order='K', subok=False, ndmin=0,
              like=None)

        Create an arra

Sprawdzenie jakie wskaźniki i funkcje są dostępne w bibliotece Pandas-TA

In [172]:
df = pd.DataFrame()
lista_wskaźników = [metoda for metoda in dir(df.ta) if not metoda.startswith('_')]

print(f"Znaleziono {len(lista_wskaźników)} wskaźników i funkcji.")
print(lista_wskaźników)

Znaleziono 157 wskaźników i funkcji.
['aberration', 'above', 'above_value', 'accbands', 'ad', 'adjusted', 'adosc', 'adx', 'alma', 'amat', 'ao', 'aobv', 'apo', 'aroon', 'atr', 'bbands', 'below', 'below_value', 'bias', 'bop', 'brar', 'categories', 'cci', 'cdl_pattern', 'cdl_z', 'cfo', 'cg', 'chop', 'cksp', 'cmf', 'cmo', 'constants', 'coppock', 'cores', 'cross', 'cross_value', 'cti', 'datetime_ordered', 'decay', 'decreasing', 'dema', 'dm', 'donchian', 'dpo', 'ebsw', 'efi', 'ema', 'entropy', 'eom', 'er', 'eri', 'exchange', 'fisher', 'fwma', 'ha', 'hilo', 'hl2', 'hlc3', 'hma', 'hwc', 'hwma', 'ichimoku', 'increasing', 'indicators', 'inertia', 'jma', 'kama', 'kc', 'kdj', 'kst', 'kurtosis', 'kvo', 'last_run', 'linreg', 'log_return', 'long_run', 'macd', 'mad', 'massi', 'mcgd', 'median', 'mfi', 'midpoint', 'midprice', 'mom', 'natr', 'nvi', 'obv', 'ohlc4', 'pdist', 'percent_return', 'pgo', 'ppo', 'psar', 'psl', 'pvi', 'pvo', 'pvol', 'pvr', 'pvt', 'pwma', 'qqe', 'qstick', 'quantile', 'reverse', 'r

### Definicja paramertów dla dla funkcji fetch() z biblioteki dukascopy_python

In [173]:
instrument = 'EUR/USD'
interval = '1DAY'
offer_side = 'B' 
start_date = datetime(2000, 1, 1)
end_date = datetime.today()

### Pobranie danych do Pandas DataFrame

In [174]:
df = dcp.fetch(instrument, interval, offer_side, start_date, end_date)

INFO:DUKASCRIPT:current timestamp :2025-08-01T02:00:00


In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7152 entries, 2003-05-04 00:00:00+00:00 to 2025-08-01 00:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    7152 non-null   float64
 1   high    7152 non-null   float64
 2   low     7152 non-null   float64
 3   close   7152 non-null   float64
 4   volume  7152 non-null   float64
dtypes: float64(5)
memory usage: 335.2 KB


Zbiór jest kompletny, zgodnie z oczekiwaniami zawiera dane do dnia poprzedzającego dzień uruchomienia skryptu.
Pora zatem na rozbudowanie zbioru i policzenie interesujących nas statystyk, wskaźników i miar.

Wskaźniki trendu

1. adx (Average Directional Index)
2. amat (Archer Moving Average Trend)
3. aroon (Aroon Indicator)
4. cti (Correlation Trend Indicator)
5. dm (Directional Movement)
6. ebsw (Even Better Sine Wave)
7. er (Efficiency Ratio)
8. ichimoku (Ichimoku Kinko Hyo)
9. linreg (Linear Regression)
10. psar (Parabolic Stop and Reverse)
11. slope (Slope)
12. supertrend (Supertrend)
13. td_seq (TD Sequential)
14. ttm_trend (TTM Trend)
15. vhf (Vertical Horizontal Filter)
16. vortex (Vortex Indicator)

In [176]:
# 1. Average Directional Index, ustawienia standardowe
# Default Inputs: length=14, lensig=lenght, scalar=100, mamode='rma', drift=1, offset=0 <- parametry do modyfikacji
df.ta.adx(append=True)

,ADX_14,DMP_14,DMN_14
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,22.819567,20.076440,27.971068
2025-07-29 00:00:00+00:00,23.094972,18.568266,32.078387
2025-07-30 00:00:00+00:00,24.348391,15.824123,37.494199


In [177]:
# 2. Archer Moving Average Trend, ustawienia standardowe
# Default Inputs: fast=8, slow=21, lookback=2, mamode='ema', offset=0 <- parametry do modyfikacji
df.ta.amat(append=True)

,AMATe_LR_8_21_2,AMATe_SR_8_21_2
timestamp,,
2003-05-04 00:00:00+00:00,0,0
2003-05-05 00:00:00+00:00,0,0
2003-05-06 00:00:00+00:00,0,0
2003-05-07 00:00:00+00:00,0,0
2003-05-08 00:00:00+00:00,0,0
...,...,...
2025-07-28 00:00:00+00:00,0,1
2025-07-29 00:00:00+00:00,0,1
2025-07-30 00:00:00+00:00,0,1


In [178]:
# 3. Aroon Indicator, ustawienia standardowe
# Default Inputs: length=14, scalar=100, talib=True, offset=0 <- parametry do modyfikacji
df.ta.aroon(append=True)

,AROOND_14,AROONU_14,AROONOSC_14
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,35.714286,78.571429,42.857143
2025-07-29 00:00:00+00:00,100.000000,71.428571,-28.571429
2025-07-30 00:00:00+00:00,100.000000,64.285714,-35.714286


In [179]:
# 4. Correlation Trend Indicator, ustawienia standardowe
# Default Inputs: length=12, offset=0 <- parametry do modyfikacji
df.ta.cti(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.547996
2025-07-29 00:00:00+00:00    0.101664
2025-07-30 00:00:00+00:00   -0.319069
2025-07-31 00:00:00+00:00   -0.587382
2025-08-01 00:00:00+00:00   -0.675145
Name: CTI_12, Length: 7152, dtype: float64

In [180]:
# 5. Directional Movement, ustawienia standardowe
# Default Inputs: length=14, mamode='rma', talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.dm(append=True)

,DMP_14,DMN_14
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,0.001523,0.002122
2025-07-29 00:00:00+00:00,0.001414,0.002443
2025-07-30 00:00:00+00:00,0.001313,0.003112


In [181]:
# 6. Even Better Sine Wave, ustawienia standardowe
# Default Inputs: length=40, bars=10, offset=0 <- parametry do modyfikacji
df.ta.ebsw(append=True)

C:\Users\mt242\AppData\Local\Temp\ipykernel_37700\2968336787.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.ta.ebsw(append=True)


timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.156943
2025-07-29 00:00:00+00:00   -0.612977
2025-07-30 00:00:00+00:00   -0.918954
2025-07-31 00:00:00+00:00   -0.984813
2025-08-01 00:00:00+00:00   -0.939068
Name: EBSW_40_10, Length: 7152, dtype: float64

In [182]:
# 7. Efficiency Ratio, ustawienia standardowe
# Default Inputs: length=10, drift=1, offset=0 <- parametry do modyfikacji
df.ta.er(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.100723
2025-07-29 00:00:00+00:00    0.156083
2025-07-30 00:00:00+00:00    0.364168
2025-07-31 00:00:00+00:00    0.394043
2025-08-01 00:00:00+00:00    0.178887
Name: ER_10, Length: 7152, dtype: float64

In [183]:
# 8. Ichimoku Kinko Hyo, ustawienia standardowe
# Default Inputs: tenkan=9, kijun=26, senkou=52, include_chikou=True, offset=0 <- parametry do modyfikacji
df.ta.ichimoku(append=True)

(                              ISA_9    ISB_26     ITS_9    IKS_26   ICS_26
 timestamp                                                                 
 2003-05-04 00:00:00+00:00       NaN       NaN       NaN       NaN  1.17337
 2003-05-05 00:00:00+00:00       NaN       NaN       NaN       NaN  1.16444
 2003-05-06 00:00:00+00:00       NaN       NaN       NaN       NaN  1.18397
 2003-05-07 00:00:00+00:00       NaN       NaN       NaN       NaN  1.16973
 2003-05-08 00:00:00+00:00       NaN       NaN       NaN       NaN  1.16989
 ...                             ...       ...       ...       ...      ...
 2025-07-28 00:00:00+00:00  1.154422  1.140950  1.168685  1.169320      NaN
 2025-07-29 00:00:00+00:00  1.156655  1.140950  1.165375  1.167430      NaN
 2025-07-30 00:00:00+00:00  1.159455  1.142790  1.159475  1.161530      NaN
 2025-07-31 00:00:00+00:00  1.161633  1.144745  1.159475  1.160540      NaN
 2025-08-01 00:00:00+00:00  1.161742  1.144745  1.159010  1.160075      NaN
 
 [7152 row

In [184]:
# 9. Linear Regression, ustawienia standardowe
# Default Inputs: length=10, offset=0 <- parametry do modyfikacji
df.ta.linreg(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    1.171663
2025-07-29 00:00:00+00:00    1.168764
2025-07-30 00:00:00+00:00    1.163186
2025-07-31 00:00:00+00:00    1.157406
2025-08-01 00:00:00+00:00    1.156071
Name: LR_14, Length: 7152, dtype: float64

In [185]:
# 10. Parabolic Stop and Reverse, ustawienia standardowe
# Default Inputs: af0=0.02, af=0.02, max_af=0.2, offset=0 <- parametry do modyfikacji
df.ta.psar(append=True)

,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,0.02,0
2003-05-05 00:00:00+00:00,NaN,1.130090,0.02,1
2003-05-06 00:00:00+00:00,1.120140,NaN,0.02,1
2003-05-07 00:00:00+00:00,1.120140,NaN,0.02,0
2003-05-08 00:00:00+00:00,1.120638,NaN,0.04,0
...,...,...,...,...
2025-07-28 00:00:00+00:00,NaN,1.178870,0.02,1
2025-07-29 00:00:00+00:00,NaN,1.178463,0.04,0
2025-07-30 00:00:00+00:00,NaN,1.177399,0.06,0


In [186]:
# 11. Slope, ustawienia standardowe
# Default Inputs: length=1, as_angle=False, to_degrees=False, vertical=None, offset=0 <- parametry do modyfikacji
df.ta.slope(append=True)

timestamp
2003-05-04 00:00:00+00:00        NaN
2003-05-05 00:00:00+00:00    0.00755
2003-05-06 00:00:00+00:00    0.01310
2003-05-07 00:00:00+00:00   -0.00740
2003-05-08 00:00:00+00:00    0.01326
                              ...   
2025-07-28 00:00:00+00:00   -0.01591
2025-07-29 00:00:00+00:00   -0.00457
2025-07-30 00:00:00+00:00   -0.01228
2025-07-31 00:00:00+00:00   -0.00061
2025-08-01 00:00:00+00:00    0.01613
Name: SLOPE_1, Length: 7152, dtype: float64

In [187]:
# 12. Supertrend, ustawienia standardowe
# Default Inputs: length=7, multiplier=3.0, offset=0 <- parametry do modyfikacji
df.ta.supertrend(append=True)

,SUPERT_7_3.0,SUPERTd_7_3.0,SUPERTl_7_3.0,SUPERTs_7_3.0
timestamp,,,,
2003-05-04 00:00:00+00:00,0.000000,1,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,1,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,1,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,1,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,1,NaN,NaN
...,...,...,...,...
2025-07-28 00:00:00+00:00,1.159619,1,1.159619,NaN
2025-07-29 00:00:00+00:00,1.179974,-1,NaN,1.179974
2025-07-30 00:00:00+00:00,1.176671,-1,NaN,1.176671


In [188]:
# 13. TD Sequential, ustawienia standardowe
# Default Inputs: asint=False, offset=0 <- parametry do modyfikacji
df.ta.td_seq(append=True)

,TD_SEQ_UP,TD_SEQ_DN
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
7147,NaN,NaN
7148,NaN,NaN
7149,NaN,NaN
7150,NaN,NaN


In [189]:
# 14. TTM Trend, ustawienia standardowe
# Default Inputs: length=6, offset=0 <- parametry do modyfikacji
df.ta.ttm_trend(append=True)

,TTM_TRND_6
timestamp,
2003-05-04 00:00:00+00:00,-1
2003-05-05 00:00:00+00:00,-1
2003-05-06 00:00:00+00:00,-1
2003-05-07 00:00:00+00:00,-1
2003-05-08 00:00:00+00:00,-1
...,...
2025-07-28 00:00:00+00:00,-1
2025-07-29 00:00:00+00:00,-1
2025-07-30 00:00:00+00:00,-1


In [190]:
# 15. Vertical Horizontal Filter, ustawienia standardowe
# Default Inputs: length=28, drift=1, offset=0 <- parametry do modyfikacji
df.ta.vhf(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.265252
2025-07-29 00:00:00+00:00    0.309263
2025-07-30 00:00:00+00:00    0.409022
2025-07-31 00:00:00+00:00    0.417447
2025-08-01 00:00:00+00:00    0.375969
Name: VHF_28, Length: 7152, dtype: float64

In [191]:
# 16. Vortex Indicator, ustawienia standardowe
# Default Inputs: length=14, drift=1, offset=0 <- parametry do modyfikacji
df.ta.vortex(append=True)

,VTXP_14,VTXM_14
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,0.912482,0.934432
2025-07-29 00:00:00+00:00,0.861477,1.065235
2025-07-30 00:00:00+00:00,0.786592,1.090566


In [192]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7152 entries, 2003-05-04 00:00:00+00:00 to 2025-08-01 00:00:00+00:00
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   open             7152 non-null   float64
 1   high             7152 non-null   float64
 2   low              7152 non-null   float64
 3   close            7152 non-null   float64
 4   volume           7152 non-null   float64
 5   ADX_14           7125 non-null   float64
 6   DMP_14           7138 non-null   float64
 7   DMN_14           7138 non-null   float64
 8   AMATe_LR_8_21_2  7152 non-null   int32  
 9   AMATe_SR_8_21_2  7152 non-null   int32  
 10  AROOND_14        7138 non-null   float64
 11  AROONU_14        7138 non-null   float64
 12  AROONOSC_14      7138 non-null   float64
 13  CTI_12           7141 non-null   float64
 14  EBSW_40_10       7113 non-null   float64
 15  ER_10            7142 non-null   float64
 16  ISA_9       

Oscylatory i wskaźniki pędu (momentum)

1. ao (Awesome Oscillator)
2. apo (Absolute Price Oscillator)
3. bias (Bias)
4. bop (Balance of Power)
5. brar (AR/BR Ratio)
6. cci (Commodity Channel Index)
7. cfo (Chande Forecast Oscillator)
8. cg (Center of Gravity)
9. cmo (Chande Momentum Oscillator)
10. coppock (Coppock Curve) <- oscylator miesięczny, trzeba dostosować parametry: 14 -> 14*21=294, 11 -> 11*21=231, 10 -> 10*21=210, 
Default Inputs: length=10, fast=11, slow=14
11. dpo (Detrended Price Oscillator)
12. eri (Elder Ray Index)
13. fisher (Fisher Transform)
14. inertia (Inertia)
15. kdj (KDJ Indicator)
16. kst (KST Oscillator)
17. macd (Moving Average Convergence Divergence)
18. mom (Momentum)
19. pgo (Pretty Good Oscillator)
20. ppo (Percentage Price Oscillator)
21. psl (Psychological Line)
22. qqe (Quantitative Qualitative Estimation)
23. qstick (QStick)
24. roc (Rate of Change)
25. rsi (Relative Strength Index)
26. rsx (Relative Strength Xtra)
27. rvgi (Relative Vigor Index)
28. smi (Stochastic Momentum Index)
29. stc (Schaff Trend Cycle)
30. stoch (Stochastic Oscillator)
31. stochrsi (Stochastic RSI)
32. thermo (Thermometer)
33. trix (Trix)
34. tsi (True Strength Index)
35. uo (Ultimate Oscillator)
36. willr (Williams' %R)

In [193]:
# 1. Awesome Oscillator, ustawienia standardowe
# Default Inputs: fast=5, slow=34, offset=0 <- parametry do modyfikacji
df.ta.ao(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.005296
2025-07-29 00:00:00+00:00    0.001341
2025-07-30 00:00:00+00:00   -0.004020
2025-07-31 00:00:00+00:00   -0.009853
2025-08-01 00:00:00+00:00   -0.015054
Name: AO_5_34, Length: 7152, dtype: float64

In [194]:
#2. Absolute Price Oscillator, ustawienia standardowe
# Default Inputs: fast=12, slow=26, mamode='sma', talib=True, offset=0 <- parametry do modyfikacji
df.ta.apo(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00   -0.003268
2025-07-29 00:00:00+00:00   -0.003063
2025-07-30 00:00:00+00:00   -0.003413
2025-07-31 00:00:00+00:00   -0.003550
2025-08-01 00:00:00+00:00   -0.003035
Name: APO_12_26, Length: 7152, dtype: float64

In [195]:
# 3. Bias, ustawienia standardowe
# Default Inputs: length=26, mamode='sma', drift=1, offset=0 <- parametry do modyfikacji
df.ta.bias(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00   -0.009928
2025-07-29 00:00:00+00:00   -0.013261
2025-07-30 00:00:00+00:00   -0.022600
2025-07-31 00:00:00+00:00   -0.021888
2025-08-01 00:00:00+00:00   -0.007371
Name: BIAS_SMA_26, Length: 7152, dtype: float64

In [196]:
# 4. Balance of Power, ustawienia standardowe
# Default Inputs: scalar=1, talib=True, offset=0 <- parametry do modyfikacji
df.ta.bop(append=True)

timestamp
2003-05-04 00:00:00+00:00   -0.584270
2003-05-05 00:00:00+00:00    0.766834
2003-05-06 00:00:00+00:00    0.736399
2003-05-07 00:00:00+00:00   -0.684310
2003-05-08 00:00:00+00:00    0.633993
                               ...   
2025-07-28 00:00:00+00:00   -0.860000
2025-07-29 00:00:00+00:00   -0.566792
2025-07-30 00:00:00+00:00   -0.715201
2025-07-31 00:00:00+00:00   -0.111314
2025-08-01 00:00:00+00:00    0.785297
Name: BOP, Length: 7152, dtype: float64

In [197]:
# 5. AR/BR Ratio, ustawienia standardowe
# Default Inputs: length=26, scalar=100, drift=1, offset=0 <- parametry do modyfikacji
df.ta.brar(append=True)

,AR_26,BR_26
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,67.683786,69.768754
2025-07-29 00:00:00+00:00,62.901947,63.305003
2025-07-30 00:00:00+00:00,53.048973,53.446514


In [198]:
# 6. Commodity Channel Index, ustawienia standardowe
# Default Inputs: length=14, c=0.015, talib=True, offset=0 <- parametry do modyfikacji
df.ta.cci(append=True)

timestamp
2003-05-04 00:00:00+00:00           NaN
2003-05-05 00:00:00+00:00           NaN
2003-05-06 00:00:00+00:00           NaN
2003-05-07 00:00:00+00:00           NaN
2003-05-08 00:00:00+00:00           NaN
                                ...    
2025-07-28 00:00:00+00:00    -44.800082
2025-07-29 00:00:00+00:00   -147.994260
2025-07-30 00:00:00+00:00   -195.555185
2025-07-31 00:00:00+00:00   -182.272667
2025-08-01 00:00:00+00:00    -85.939144
Name: CCI_14_0.015, Length: 7152, dtype: float64

In [199]:
# 7. Chande Forecast Oscillator, ustawienia standardowe
# Default Inputs: length=9, scalar=100, drift=1, offset=0 <- parametry do modyfikacji
df.ta.cfo(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00   -1.118345
2025-07-29 00:00:00+00:00   -0.876357
2025-07-30 00:00:00+00:00   -0.963246
2025-07-31 00:00:00+00:00   -0.243688
2025-08-01 00:00:00+00:00    1.148810
Name: CFO_9, Length: 7152, dtype: float64

In [200]:
# 8. Center of Gravity, ustawienia standardowe
# Default Inputs: length=10, offset=0 <- parametry do modyfikacji
df.ta.cg(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00   -5.493409
2025-07-29 00:00:00+00:00   -5.502445
2025-07-30 00:00:00+00:00   -5.515420
2025-07-31 00:00:00+00:00   -5.526610
2025-08-01 00:00:00+00:00   -5.526706
Name: CG_10, Length: 7152, dtype: float64

In [201]:
# 9. Chande Momentum Oscillator, ustawienia standardowe
# Default Inputs: length=14, scalar=100, talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.cmo(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00   -16.135899
2025-07-29 00:00:00+00:00   -23.509399
2025-07-30 00:00:00+00:00   -39.023509
2025-07-31 00:00:00+00:00   -39.678010
2025-08-01 00:00:00+00:00    -6.978924
Name: CMO_14, Length: 7152, dtype: float64

In [202]:
# 10. Coppock Curve, ustawienia niestandardowe
# Default Inputs: length=10, fast=11, slow=14 <- oscylator miesięczy
# Dostosowanie oscylatora do interwały dziennego: 
# length=10 -> 10*21=210, fast=11 -> 11*21=231, slow=14 -> 14*21=294
# Standardowe parametry: offset=0
df.ta.coppock(length=210, fast=231, slow=294, append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    6.522328
2025-07-29 00:00:00+00:00    6.603898
2025-07-30 00:00:00+00:00    6.657408
2025-07-31 00:00:00+00:00    6.712087
2025-08-01 00:00:00+00:00    6.782220
Name: COPC_231_294_210, Length: 7152, dtype: float64

In [203]:
# 11. Detrended Price Oscillator, ustawienia standardowe
# Default Inputs: length=1, centered=True, offset=0 <- parametry do modyfikacji
df.ta.dpo(append=True)

timestamp
2003-05-04 00:00:00+00:00   NaN
2003-05-05 00:00:00+00:00   NaN
2003-05-06 00:00:00+00:00   NaN
2003-05-07 00:00:00+00:00   NaN
2003-05-08 00:00:00+00:00   NaN
                             ..
2025-07-28 00:00:00+00:00   NaN
2025-07-29 00:00:00+00:00   NaN
2025-07-30 00:00:00+00:00   NaN
2025-07-31 00:00:00+00:00   NaN
2025-08-01 00:00:00+00:00   NaN
Name: DPO_20, Length: 7152, dtype: float64

In [204]:
# 12. Elder Ray Index, ustawienia standardowe
# Default Inputs: length=14, offset=0 <- parametry do modyfikacji
df.ta.eri(append=True)

,BULLP_13,BEARP_13
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,0.007494,-0.011006
2025-07-29 00:00:00+00:00,-0.007552,-0.015562
2025-07-30 00:00:00+00:00,-0.006669,-0.023839


In [205]:
# 13. Fisher Transform, ustawienia standardowe
# Default Inputs: length=9, signal=1, offset=0 <- parametry do modyfikacji
df.ta.fisher(append=True)

,FISHERT_9_1,FISHERTs_9_1
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,1.239643,1.573155
2025-07-29 00:00:00+00:00,0.574148,1.239643
2025-07-30 00:00:00+00:00,-0.090478,0.574148


In [206]:
# 14. Inertia, ustawienia standardowe
# Default Inputs: length=20, rvi_length=14, scalar=100, refined=False, thirds=False, mamode='ema, drift=1, offset=0 <- parametry do modyfikacji
df.ta.inertia(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00    53.314259
2025-07-29 00:00:00+00:00    52.676196
2025-07-30 00:00:00+00:00    50.228606
2025-07-31 00:00:00+00:00    46.410710
2025-08-01 00:00:00+00:00    45.766247
Name: INERTIA_20_14, Length: 7152, dtype: float64

In [207]:
# 15. KDJ Indicator, ustawienia standardowe
# Default Inputs: length=9, signal=3, offset=0 <- parametry do modyfikacji
df.ta.kdj(append=True)

,K_9_3,D_9_3,J_9_3
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,55.529884,66.152728,34.284198
2025-07-29 00:00:00+00:00,40.947303,57.750920,7.340071
2025-07-30 00:00:00+00:00,29.618388,48.373409,-7.891654


In [208]:
# 16. KST Oscillator, ustawienia standardowe
# Default Inputs: roc1=10, roc2=15, roc3=20, roc4=30, sma1=10, sma2=10, sma3=10, sma4=15, signal=9, drift=1, offset=0 <- parametry do modyfikacji
df.ta.kst(append=True)

,KST_10_15_20_30_10_10_10_15,KSTs_9
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,390.809066,690.970738
2025-07-29 00:00:00+00:00,263.734945,584.107089
2025-07-30 00:00:00+00:00,58.998438,482.632269


In [209]:
# 17. Moving Average Convergence Divergence, ustawienia standardowe
# Default Inputs: fast=12, slow=26, signal=9, talib=True, offset=0 <- parametry do modyfikacji
df.ta.macd(append=True)

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,0.002037,-0.000784,0.002821
2025-07-29 00:00:00+00:00,0.000729,-0.001673,0.002403
2025-07-30 00:00:00+00:00,-0.001283,-0.002948,0.001666


In [210]:
# 18. Momentum, ustawienia standardowe
# Default Inputs: length=1, talib=True, offset=0 <- parametry do modyfikacji
df.ta.mom(append=True)

timestamp
2003-05-04 00:00:00+00:00        NaN
2003-05-05 00:00:00+00:00        NaN
2003-05-06 00:00:00+00:00        NaN
2003-05-07 00:00:00+00:00        NaN
2003-05-08 00:00:00+00:00        NaN
                              ...   
2025-07-28 00:00:00+00:00   -0.00390
2025-07-29 00:00:00+00:00   -0.00644
2025-07-30 00:00:00+00:00   -0.01929
2025-07-31 00:00:00+00:00   -0.02077
2025-08-01 00:00:00+00:00   -0.01115
Name: MOM_10, Length: 7152, dtype: float64

In [211]:
# 19. Pretty Good Oscillator, ustawienia standardowe
# Default Inputs: length=14, offset=0 <- parametry do modyfikacji
df.ta.pgo(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00   -1.146790
2025-07-29 00:00:00+00:00   -1.637899
2025-07-30 00:00:00+00:00   -3.037309
2025-07-31 00:00:00+00:00   -2.904610
2025-08-01 00:00:00+00:00   -0.684725
Name: PGO_14, Length: 7152, dtype: float64

In [212]:
help(ta.ppo)

Help on function ppo in module pandas_ta.momentum.ppo:

ppo(close, fast=None, slow=None, signal=None, scalar=None, mamode=None, talib=None, offset=None, **kwargs)
    Percentage Price Oscillator (PPO)

    The Percentage Price Oscillator is similar to MACD in measuring momentum.

    Sources:
        https://www.tradingview.com/wiki/MACD_(Moving_Average_Convergence/Divergence)

    Calculation:
        Default Inputs:
            fast=12, slow=26
        SMA = Simple Moving Average
        EMA = Exponential Moving Average
        fast_sma = SMA(close, fast)
        slow_sma = SMA(close, slow)
        PPO = 100 * (fast_sma - slow_sma) / slow_sma
        Signal = EMA(PPO, signal)
        Histogram = PPO - Signal

    Args:
        close(pandas.Series): Series of 'close's
        fast(int): The short period. Default: 12
        slow(int): The long period. Default: 26
        signal(int): The signal period. Default: 9
        scalar (float): How much to magnify. Default: 100
        mamode

In [213]:
# 20. Percentage Price Oscillator, ustawienia standardowe
# Default Inputs: fast=12, slow=26, signal=9, scalar=100, mamode='sma', talib=True, offset=0 <- parametry do modyfikacji
df.ta.ppo(append=True)

,PPO_12_26_9,PPOh_12_26_9,PPOs_12_26_9
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,-0.279056,-0.126631,-0.152424
2025-07-29 00:00:00+00:00,-0.261638,-0.087371,-0.174267
2025-07-30 00:00:00+00:00,-0.291909,-0.094114,-0.197796


In [214]:
# 21. Psychological Line, ustawienia standardowe
# Default Inputs: length=12, scalar=100, drift=1, offset=0 <- parametry do modyfikacji
df.ta.psl(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00    58.333333
2025-07-29 00:00:00+00:00    58.333333
2025-07-30 00:00:00+00:00    50.000000
2025-07-31 00:00:00+00:00    50.000000
2025-08-01 00:00:00+00:00    50.000000
Name: PSL_12, Length: 7152, dtype: float64

In [215]:
# 22. Quantitative Qualitative Estimation, ustawienia standardowe
# Default Inputs: length=14, smooth=5, factor=4.236, mamode='sma', drift=1, offset=0 <- parametry do modyfikacji
df.ta.qqe(append=True)

,QQE_14_5_4.236,QQE_14_5_4.236_RSIMA,QQEl_14_5_4.236,QQEs_14_5_4.236
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2025-07-28 00:00:00+00:00,53.464015,53.792500,53.464015,NaN
2025-07-29 00:00:00+00:00,55.179182,48.610100,NaN,55.179182
2025-07-30 00:00:00+00:00,49.396291,42.569482,NaN,49.396291


In [216]:
# 23. QStick, ustawienia standardowe
# Default Inputs: length=10, offset=0 <- parametry do modyfikacji
df.ta.qstick(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00   -0.000584
2025-07-29 00:00:00+00:00   -0.000835
2025-07-30 00:00:00+00:00   -0.002119
2025-07-31 00:00:00+00:00   -0.002238
2025-08-01 00:00:00+00:00   -0.001274
Name: QS_10, Length: 7152, dtype: float64

In [217]:
# 24. Rate of Change, ustawienia standardowe
# Default Inputs: length=1, scalar=100, talib=True, offset=0 <- parametry do modyfikacji
df.ta.roc(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00   -0.335187
2025-07-29 00:00:00+00:00   -0.554455
2025-07-30 00:00:00+00:00   -1.659969
2025-07-31 00:00:00+00:00   -1.785991
2025-08-01 00:00:00+00:00   -0.953440
Name: ROC_10, Length: 7152, dtype: float64

In [218]:
# 25. Relative Strength Index, ustawienia standardowe
# Default Inputs: length=14, scalar=100, talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.rsi(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00    41.932051
2025-07-29 00:00:00+00:00    38.245301
2025-07-30 00:00:00+00:00    30.488246
2025-07-31 00:00:00+00:00    30.160995
2025-08-01 00:00:00+00:00    46.510538
Name: RSI_14, Length: 7152, dtype: float64

In [219]:
# 26. Relative Strength Xtra, ustawienia standardowe
# Default Inputs: length=14, drift=1, offset=0 <- parametry do modyfikacji
df.ta.rsx(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00    56.084785
2025-07-29 00:00:00+00:00    49.533834
2025-07-30 00:00:00+00:00    40.887112
2025-07-31 00:00:00+00:00    33.789413
2025-08-01 00:00:00+00:00    32.979340
Name: RSX_14, Length: 7152, dtype: float64

In [220]:
# 27. Relative Vigor Index, ustawienia standardowe
# Default Inputs: length=14, swma_length=4, offset=0 <- parametry do modyfikacji
df.ta.rvgi(append=True)

,RVGI_14_4,RVGIs_14_4
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,0.017264,0.027903
2025-07-29 00:00:00+00:00,-0.034090,0.018932
2025-07-30 00:00:00+00:00,-0.105943,-0.016505


In [221]:
# 28. Stochastic Momentum Index, ustawienia standardowe
# Default Inputs: fast=5, slow=20, signal=5, scalar=1, offset=0 <- parametry do modyfikacji
df.ta.smi(append=True)

,SMI_5_20_5,SMIs_5_20_5,SMIo_5_20_5
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,0.017446,0.107658,-0.090212
2025-07-29 00:00:00+00:00,-0.118530,0.032262,-0.150792
2025-07-30 00:00:00+00:00,-0.274861,-0.070112,-0.204748


In [222]:
# 29. Schaff Trend Cycle, ustawienia standardowe
# Default Inputs: tclength=10, fast=12, slow=26, factor=0.5, offset=0 <- parametry do modyfikacji
df.ta.stc(append=True)

C:\Users\mt242\AppData\Local\Temp\ipykernel_37700\4005191877.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.ta.stc(append=True)


,STC_10_12_26_0.5,STCmacd_10_12_26_0.5,STCstoch_10_12_26_0.5
timestamp,,,
2003-05-04 00:00:00+00:00,0.000000,NaN,0.000000
2003-05-05 00:00:00+00:00,0.000000,NaN,0.000000
2003-05-06 00:00:00+00:00,0.000000,NaN,0.000000
2003-05-07 00:00:00+00:00,0.000000,NaN,0.000000
2003-05-08 00:00:00+00:00,0.000000,NaN,0.000000
...,...,...,...
2025-07-28 00:00:00+00:00,86.117904,0.002037,57.211242
2025-07-29 00:00:00+00:00,61.830227,0.000729,28.605621
2025-07-30 00:00:00+00:00,40.285313,-0.001283,14.302810


In [223]:
# 30. Stochastic Oscillator, ustawienia standardowe
# Default Inputs: k=14, d=3, smooth_k=3, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.stoch(append=True)

,STOCHk_14_3_3,STOCHd_14_3_3
timestamp,,
2003-05-19 00:00:00+00:00,NaN,NaN
2003-05-20 00:00:00+00:00,NaN,NaN
2003-05-21 00:00:00+00:00,86.540110,NaN
2003-05-22 00:00:00+00:00,88.448066,NaN
2003-05-23 00:00:00+00:00,88.512464,87.833546
...,...,...
2025-07-28 00:00:00+00:00,60.534252,76.995219
2025-07-29 00:00:00+00:00,38.179858,60.656152
2025-07-30 00:00:00+00:00,11.949132,36.887747


In [224]:
# 31. Stochastic RSI, ustawienia standardowe
# Default Inputs: length=14, rsi_length=14, k=3, d=3, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.stochrsi(append=True)

,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,51.684064,72.599009
2025-07-29 00:00:00+00:00,27.260249,52.988084
2025-07-30 00:00:00+00:00,0.000000,26.314771


In [225]:
# 32. Thermometer, ustawienia standardowe
# Default Inputs: length=20, long=2, short=0.5, mamode='ema', drift=1, offset=0 <- parametry do modyfikacji
df.ta.thermo(append=True)

,THERMO_20_2_0.5,THERMOma_20_2_0.5,THERMOl_20_2_0.5,THERMOs_20_2_0.5
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,0,0
2003-05-05 00:00:00+00:00,0.00671,NaN,0,0
2003-05-06 00:00:00+00:00,0.01497,NaN,0,0
2003-05-07 00:00:00+00:00,0.00542,NaN,0,0
2003-05-08 00:00:00+00:00,0.00754,NaN,0,0
...,...,...,...,...
2025-07-28 00:00:00+00:00,0.01657,0.005556,0,1
2025-07-29 00:00:00+00:00,0.01711,0.006656,0,1
2025-07-30 00:00:00+00:00,0.01180,0.007146,1,1


In [226]:
# 33. Trix, ustawienia standardowe
# Default Inputs: length=18, signal=9, scalar=100, drift=1, offset=0 <- parametry do modyfikacji
df.ta.trix(append=True)

,TRIX_30_9,TRIXs_30_9
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,0.066583,0.071504
2025-07-29 00:00:00+00:00,0.064837,0.070160
2025-07-30 00:00:00+00:00,0.062499,0.068726


In [227]:
# 34. True Strength Index, ustawienia standardowe
# Default Inputs: fast=13, slow=25, signal=13, scalar=100, mamode='ema', drift=1, offset=0 <- parametry do modyfikacji
df.ta.tsi(append=True)

,TSI_13_25_13,TSIs_13_25_13
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,9.883594,14.950402
2025-07-29 00:00:00+00:00,3.614293,13.330958
2025-07-30 00:00:00+00:00,-5.168391,10.688194


In [228]:
# 35. Ultimate Oscillator, ustawienia standardowe
# Default Inputs: fast=7, medium=14, slow=28, fast_w=4.0, medium_w=2.0, slow_w=1.0, talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.uo(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00    48.773097
2025-07-29 00:00:00+00:00    44.628806
2025-07-30 00:00:00+00:00    37.086152
2025-07-31 00:00:00+00:00    33.705838
2025-08-01 00:00:00+00:00    45.767387
Name: UO_7_14_28, Length: 7152, dtype: float64

In [229]:
# 36. Williams' %R, ustawienia standardowe
# Default Inputs: length=14, talib=True, offset=0 <- parametry do modyfikacji
df.ta.willr(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00   -82.895304
2025-07-29 00:00:00+00:00   -88.217858
2025-07-30 00:00:00+00:00   -93.039443
2025-07-31 00:00:00+00:00   -94.612013
2025-08-01 00:00:00+00:00   -51.787513
Name: WILLR_14, Length: 7152, dtype: float64

In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7152 entries, 2003-05-04 00:00:00+00:00 to 2025-08-01 00:00:00+00:00
Data columns (total 98 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   open                         7152 non-null   float64
 1   high                         7152 non-null   float64
 2   low                          7152 non-null   float64
 3   close                        7152 non-null   float64
 4   volume                       7152 non-null   float64
 5   ADX_14                       7125 non-null   float64
 6   DMP_14                       7138 non-null   float64
 7   DMN_14                       7138 non-null   float64
 8   AMATe_LR_8_21_2              7152 non-null   int32  
 9   AMATe_SR_8_21_2              7152 non-null   int32  
 10  AROOND_14                    7138 non-null   float64
 11  AROONU_14                    7138 non-null   float64
 12  AROONOSC_14                 

Wskaźniki zmienności i kanały

1. aberration (Aberration)
2. accbands (Acceleration Bands)
3. atr (Average True Range)
4. bbands (Bollinger Bands)
5. chop (Choppiness Index)
6. cksp (Chande Kroll Stop)
7. donchian (Donchian Channels)
8. entropy (Information Entropy)
9. hilo (High-Low Channels)
10. kc (Keltner Channels)
11. massi (Mass Index)
12. natr (Normalized Average True Range)
13. rvi (Relative Volatility Index)
14. squeeze (TTM Squeeze)
15. squeeze_pro (TTM Squeeze Pro)
16. tos_stdevall (thinkorswim Standard Deviation All)
17. true_range (True Range)
18. ui (Ulcer Index)

In [149]:
# 1. Aberration, ustawienia standardowe
# Default Inputs: length=5, atr_length=15, offset=0 <- parametry do modyfikacji
df.ta.aberration(append=True)

,ABER_ZG_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ATR_5_15
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,1.133375,NaN,NaN,NaN
...,...,...,...,...
2025-07-28 00:00:00+00:00,1.173106,1.180681,1.165531,0.007575
2025-07-29 00:00:00+00:00,1.169141,1.176745,1.161537,0.007604
2025-07-30 00:00:00+00:00,1.163325,1.171566,1.155083,0.008242


In [150]:
# 2. Acceleration Bands, ustawienia standardowe
# Default Inputs: length=10, c=4, mamode='sma', drift=1, offset=0 <- parametry do modyfikacji
df.ta.accbands(append=True)

,ACCBL_20,ACCBM_20,ACCBU_20
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,1.151726,1.169352,1.187469
2025-07-29 00:00:00+00:00,1.149749,1.168205,1.187262
2025-07-30 00:00:00+00:00,1.147643,1.166663,1.186866


In [151]:
# 3. Average True Range, ustawienia standardowe
# Default Inputs: length=14, mamode='rma', talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.atr(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.007586
2025-07-29 00:00:00+00:00    0.007616
2025-07-30 00:00:00+00:00    0.008299
2025-07-31 00:00:00+00:00    0.008097
2025-08-01 00:00:00+00:00    0.008986
Name: ATRr_14, Length: 7152, dtype: float64

In [152]:
# 4. Bollinger Bands, ustawienia standardowe
# Default Inputs: length=5, std=2, ddof=0, mamode='sma', talib=True, offset=0 <- parametry do modyfikacji
df.ta.bbands(append=True)

,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
timestamp,,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,1.116562,1.135282,1.154002,3.297886,0.845029
...,...,...,...,...,...
2025-07-28 00:00:00+00:00,1.159452,1.172382,1.185312,2.205717,0.006872
2025-07-29 00:00:00+00:00,1.150370,1.167928,1.185486,3.006677,0.133555
2025-07-30 00:00:00+00:00,1.136936,1.161394,1.185852,4.211789,0.119466


In [153]:
# 5. Choppiness Index, ustawienia standardowe
# Default Inputs: length=14, atr_length=1, ln=False, scalar=100, drift=1, offset=0 <- parametry do modyfikacji
df.ta.chop(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00    57.606553
2025-07-29 00:00:00+00:00    53.837323
2025-07-30 00:00:00+00:00    44.203085
2025-07-31 00:00:00+00:00    42.808689
2025-08-01 00:00:00+00:00    43.350934
Name: CHOP_14_1_100, Length: 7152, dtype: float64

In [154]:
# 6. Chande Kroll Stop, ustawienia standardowe
# Default Inputs: p=10, x=1, q=9, tvmode=True, offset=0 <- parametry do modyfikacji
df.ta.cksp(append=True)

,CKSPl_10_3_20,CKSPs_10_3_20
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,1.163253,1.175247
2025-07-29 00:00:00+00:00,1.162446,1.175143
2025-07-30 00:00:00+00:00,1.162446,1.166168


In [155]:
# 7. Donchian Channels, ustawienia standardowe
# Default Inputs: lower_length=20, upper_length=20, offset=0 <- parametry do modyfikacji
df.ta.donchian(append=True)

,DCL_20_20,DCM_20_20,DCU_20_20
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,1.15566,1.167335,1.17901
2025-07-29 00:00:00+00:00,1.15188,1.165445,1.17901
2025-07-30 00:00:00+00:00,1.14008,1.159475,1.17887


In [156]:
# 8. Information Entropy, ustawienia standardowe
# Default Inputs: length=10, base=2, offset=0 <- parametry do modyfikacji
df.ta.entropy(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    3.325034
2025-07-29 00:00:00+00:00    3.323890
2025-07-30 00:00:00+00:00    3.320776
2025-07-31 00:00:00+00:00    3.317604
2025-08-01 00:00:00+00:00    3.316120
Name: ENTP_10, Length: 7152, dtype: float64

In [157]:
# 9. High-Low Channels, ustawienia standardowe
# Default Inputs: high_length=13, low_length=21, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.hilo(append=True)

,HILO_13_21,HILOl_13_21,HILOs_13_21
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,1.172289,NaN,1.172289
2025-07-29 00:00:00+00:00,1.171528,NaN,1.171528
2025-07-30 00:00:00+00:00,1.170605,NaN,1.170605


In [158]:
# 10. Keltner Channels, ustawienia standardowe
# Default Inputs: length=20, scalar=2, mamode='ema', offset=0 <- parametry do modyfikacji
df.ta.kc(append=True)

,KCLe_20_2,KCBe_20_2,KCUe_20_2
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,1.153341,1.168734,1.184128
2025-07-29 00:00:00+00:00,1.151979,1.167432,1.182885
2025-07-30 00:00:00+00:00,1.147832,1.165084,1.182336


In [159]:
# 11. Mass Index, ustawienia standardowe
# Default Inputs: fast=9, slow=25, offset=0 <- parametry do modyfikacji
df.ta.massi(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00    24.544247
2025-07-29 00:00:00+00:00    24.753080
2025-07-30 00:00:00+00:00    25.100503
2025-07-31 00:00:00+00:00    25.298507
2025-08-01 00:00:00+00:00    25.605867
Name: MASSI_9_25, Length: 7152, dtype: float64

In [160]:
# 12. Normalized Average True Range, ustawienia standardowe
# Default Inputs: length=20, scalar=100, mamode='ema', talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.natr(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.686464
2025-07-29 00:00:00+00:00    0.689752
2025-07-30 00:00:00+00:00    0.804539
2025-07-31 00:00:00+00:00    0.761611
2025-08-01 00:00:00+00:00    0.887310
Name: NATR_14, Length: 7152, dtype: float64

In [161]:
# 13. Relative Volatility Index, ustawienia standardowe
# Default Inputs: length=14, scalar=100, refined=False, thirds=False, mamode='ema', drift=1, offset=0 <- parametry do modyfikacji
df.ta.rvi(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00    49.893494
2025-07-29 00:00:00+00:00    41.993566
2025-07-30 00:00:00+00:00    33.707001
2025-07-31 00:00:00+00:00    26.550435
2025-08-01 00:00:00+00:00    41.104084
Name: RVI_14, Length: 7152, dtype: float64

In [162]:
# 14. TTM Squeeze, ustawienia standardowe
# Default Inputs: bb_length=20, bb_std=2, kc_length=20, kc_scalar=1.5, mom_length=12, mom_smooth=6, use_tr=True, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.squeeze(append=True)

,SQZ_20_2.0_20_1.5,SQZ_ON,SQZ_OFF,SQZ_NO
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,0,0,1
2003-05-05 00:00:00+00:00,NaN,0,0,1
2003-05-06 00:00:00+00:00,NaN,0,0,1
2003-05-07 00:00:00+00:00,NaN,0,0,1
2003-05-08 00:00:00+00:00,NaN,0,0,1
...,...,...,...,...
2025-07-28 00:00:00+00:00,0.002693,0,1,0
2025-07-29 00:00:00+00:00,0.001512,0,1,0
2025-07-30 00:00:00+00:00,-0.002538,0,1,0


In [163]:
# 15. TTM Squeeze Pro, ustawienia standardowe
# Default Inputs: bb_length=20, bb_std=2, kc_length=20, kc_scalar_wide=2, kc_scalar_normal=1.5, kc_scalar_narrow=1, mom_length=12, mom_smooth=6, use_tr=True, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.squeeze_pro(append=True)

,SQZPRO_20_2.0_20_2_1.5_1,SQZPRO_ON_WIDE,SQZPRO_ON_NORMAL,SQZPRO_ON_NARROW,SQZPRO_OFF,SQZPRO_NO
timestamp,,,,,,
2003-05-04 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-05 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-06 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-07 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-08 00:00:00+00:00,NaN,0,0,0,0,1
...,...,...,...,...,...,...
2025-07-28 00:00:00+00:00,0.002693,1,0,0,0,0
2025-07-29 00:00:00+00:00,0.001512,1,0,0,0,0
2025-07-30 00:00:00+00:00,-0.002538,0,0,0,1,0


In [164]:
# 16. Think or Swim Standard Deviation All, ustawienia standardowe
# Default Inputs: length=None (means All), stds=[1,2,3], ddof=1, offset=0 <- parametry do modyfikacji
df.ta.tos_stdevall(append=True)

,TOS_STDEVALL_LR,TOS_STDEVALL_L_1,TOS_STDEVALL_U_1,TOS_STDEVALL_L_2,TOS_STDEVALL_U_2,TOS_STDEVALL_L_3,TOS_STDEVALL_U_3
timestamp,,,,,,,
2003-05-04 00:00:00+00:00,1.369040,1.241146,1.496935,1.113252,1.624829,0.985357,1.752723
2003-05-05 00:00:00+00:00,1.369001,1.241106,1.496895,1.113212,1.624789,0.985318,1.752684
2003-05-06 00:00:00+00:00,1.368961,1.241067,1.496856,1.113173,1.624750,0.985278,1.752644
2003-05-07 00:00:00+00:00,1.368922,1.241027,1.496816,1.113133,1.624710,0.985239,1.752605
2003-05-08 00:00:00+00:00,1.368882,1.240988,1.496777,1.113094,1.624671,0.985199,1.752565
...,...,...,...,...,...,...,...
2025-07-28 00:00:00+00:00,1.086748,0.958854,1.214642,0.830959,1.342537,0.703065,1.470431
2025-07-29 00:00:00+00:00,1.086708,0.958814,1.214603,0.830920,1.342497,0.703026,1.470391
2025-07-30 00:00:00+00:00,1.086669,0.958775,1.214563,0.830880,1.342458,0.702986,1.470352


In [165]:
# 17. True Range, ustawienia standardowe
# Default Inputs: talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.true_range(append=True)

timestamp
2003-05-04 00:00:00+00:00        NaN
2003-05-05 00:00:00+00:00    0.00995
2003-05-06 00:00:00+00:00    0.01783
2003-05-07 00:00:00+00:00    0.01058
2003-05-08 00:00:00+00:00    0.02071
                              ...   
2025-07-28 00:00:00+00:00    0.01850
2025-07-29 00:00:00+00:00    0.00801
2025-07-30 00:00:00+00:00    0.01717
2025-07-31 00:00:00+00:00    0.00548
2025-08-01 00:00:00+00:00    0.02054
Name: TRUERANGE_1, Length: 7152, dtype: float64

In [166]:
# 18. Ulcer Index, ustawienia standardowe
# Default Inputs: length=14, scalar=100, offset=0 <- parametry do modyfikacji
df.ta.ui(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    1.074635
2025-07-29 00:00:00+00:00    1.154609
2025-07-30 00:00:00+00:00    1.359337
2025-07-31 00:00:00+00:00    1.511260
2025-08-01 00:00:00+00:00    1.524112
Name: UI_14, Length: 7152, dtype: float64

In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7152 entries, 2003-05-04 00:00:00+00:00 to 2025-08-01 00:00:00+00:00
Data columns (total 85 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   open                      7152 non-null   float64
 1   high                      7152 non-null   float64
 2   low                       7152 non-null   float64
 3   close                     7152 non-null   float64
 4   volume                    7152 non-null   float64
 5   ADX_14                    7125 non-null   float64
 6   DMP_14                    7138 non-null   float64
 7   DMN_14                    7138 non-null   float64
 8   AMATe_LR_8_21_2           7152 non-null   int32  
 9   AMATe_SR_8_21_2           7152 non-null   int32  
 10  AROOND_14                 7138 non-null   float64
 11  AROONU_14                 7138 non-null   float64
 12  AROONOSC_14               7138 non-null   float64
 13  CTI_12         

Wskaźniki wolumenu

1. ad (Accumulation/Distribution Line)
2. adosc (Chaikin Oscillator)
3. aobv (Adaptive On-Balance Volume)
4. cmf (Chaikin Money Flow)
5. efi (Elder's Force Index)
6. eom (Ease of Movement)
7. kvo (Klinger Volume Oscillator)
8. mfi (Money Flow Index)
9. nvi (Negative Volume Index)
10. obv (On-Balance Volume)
11. pvi (Positive Volume Index)
12. pvo (Percentage Volume Oscillator)
13. pvol (Price-Volume)
14. pvr (Price-Volume Rank)
15. pvt (Price and Volume Trend)
16. vp (Volume Profile)
17. vwap (Volume-Weighted Average Price)
18. vwma (Volume-Weighted Moving Average)

In [ ]:
help(ta.ad)

In [ ]:
# 1. Accumulation/Distribution Line, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.ad(append=True)

In [ ]:
help(ta.adosc)

In [ ]:
# 2. Chaikin Oscillator, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.adosc(append=True)

In [ ]:
help(ta.aobv)

In [ ]:
# 3. Adaptive On-Balance Volume, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.aobv(append=True)

In [ ]:
help(ta.cmf)

In [ ]:
# 4. Chaikin Money Flow, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.cmf(append=True)

In [ ]:
help(ta.efi)

In [ ]:
# 5. Elder's Force Index, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.efi(append=True)

In [ ]:
help(ta.eom)

In [ ]:
# 6. Ease of Movement, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.eom(append=True)

In [ ]:
help(ta.kvo)

In [ ]:
# 7. Klinger Volume Oscillator, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.kvo(append=True)

In [ ]:
help(ta.mfi)

In [ ]:
# 8. Money Flow Index, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.mfi(append=True)

In [ ]:
help(ta.nvi)

In [ ]:
# 9. Negative Volume Index, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.nvi(append=True)

In [ ]:
help(ta.obv)

In [ ]:
# 10. On-Balance Volume, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.obv(append=True)

In [ ]:
help(ta.pvi)

In [ ]:
# 11. Positive Volume Index, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.pvi(append=True)

In [ ]:
help(ta.pvo)

In [ ]:
# 12. Percentage Volume Oscillator, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.pvo(append=True)

In [ ]:
help(ta.pvol)

In [ ]:
# 13. Price-Volume, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.pvol(append=True)

In [ ]:
help(ta.pvr)

In [ ]:
# 14. Price-Volume Rank, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.pvr(append=True)

In [ ]:
help(ta.pvt)

In [ ]:
# 15. Price and Volume Trend, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.pvt(append=True)

In [ ]:
help(ta.vp)

In [ ]:
# 16. Volume Profile, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.vp(append=True)

In [ ]:
help(ta.vmap)

In [ ]:
# 17. Volume-Weighted Average Price, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.vwap(append=True)

In [ ]:
help(ta.vwma)

In [ ]:
# 18. Volume-Weighted Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.vwma(append=True)

In [ ]:
df.info()

Funkcje rozpoznające formacje świecowe

1. cdl_pattern (Candlestick Pattern)
2. cdl_z (Candlestick Z-Score)

In [ ]:
help(ta.cdl_pattern)

In [ ]:
# 1. Candlestick Pattern, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.cdl_pattern(append=True)

In [ ]:
help(ta.cdl_z)

In [ ]:
# 2. Candlestick Z-Score, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.cdl_z(append=True)

Średnie Kroczące, filtry i funkcje wygładzające średnie

1. alma (Arnaud Legoux Moving Average)
2. dema (Double Exponential Moving Average)
3. ema (Exponential Moving Average)
4. fwma (Fibonacci's Weighted Moving Average)
5. hma (Hull Moving Average)
6. hwma (Holt-Winters Moving Average)
7. jma (Jurik Moving Average)
8. kama (Kaufman's Adaptive Moving Average)
9. mcgd (McGinley Dynamic)
10. pwma (Pascal's Weighted Moving Average)
11. rma (Running Moving Average)
12. sinwma (Sine Weighted Moving Average)
13. sma (Simple Moving Average)
14. ssf (Ehlers Super Smoother Filter)
15. swma (Symmetric Weighted Moving Average)
16. t3 (T3 Moving Average)
17. tema (Triple Exponential Moving Average)
18. trima (Triangular Moving Average)
19. vidya (Variable Index Dynamic Average)
20. wma (Weighted Moving Average)
21. zlma (Zero Lag Moving Average)

In [ ]:
help(ta.alma)

In [ ]:
# 1. Arnaud Legoux Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.alma(append=True)

In [ ]:
help(ta.dema)

In [ ]:
# 2. Double Exponential Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.dema(append=True)

In [ ]:
help(ta.ema)

In [ ]:
# 3. Exponential Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.ema(append=True)

In [ ]:
help(ta.fwma)

In [ ]:
# 4. Fibonacci's Weighted Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.fwma(append=True)

In [ ]:
help(ta.hma)

In [ ]:
# 5. Hull Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.hma(append=True)

In [ ]:
help(ta.hwma)

In [ ]:
# 6. Holt-Winters Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.hwma(append=True)

In [ ]:
help(ta.jma)

In [ ]:
# 7. Jurik Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.jma(append=True)

In [ ]:
help(ta.kama)

In [ ]:
# 8. Kaufman's Adaptive Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.kama(append=True)

In [ ]:
help(ta.mcgd)

In [ ]:
# 9. McGinley Dynamic, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.mcgd(append=True)

In [ ]:
help(ta.pwma)

In [ ]:
# 10. Pascal's Weighted Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.pwma(append=True)

In [ ]:
help(ta.rma)

In [ ]:
# 11. Running Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.rma(append=True)

In [ ]:
help(ta.sinwma)

In [ ]:
# 12. Sine Weighted Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.sinwma(append=True)

In [ ]:
help(ta.sma)

In [ ]:
# 13. Simple Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.sma(append=True)

In [ ]:
help(ta.ssf)

In [ ]:
# 14. Ehlers Super Smoother Filter, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.ssf(append=True)

In [ ]:
help(ta.swma)

In [ ]:
# 15. Symmetric Weighted Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.swma(append=True)

In [ ]:
help(ta.t3)

In [ ]:
# 16. T3 Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.t3(append=True)

In [ ]:
help(ta.tema)

In [ ]:
# 17. Triple Exponential Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.tema(append=True)

In [ ]:
help(ta.trima)

In [ ]:
# 18. Triangular Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.trima(append=True)

In [ ]:
help(ta.vidya)

In [ ]:
# 19. Variable Index Dynamic Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.vidya(append=True)

In [ ]:
help(ta.wma)

In [ ]:
# 20. Weighted Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.wma(append=True)

In [ ]:
help(ta.zlma)

In [ ]:
# 21. Zero Lag Moving Average, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.zlma(append=True)

In [ ]:
df.info()

Miary statystyczne i pomocnicze

1. decay (Decay)
2. hwc / wcp (Heavily Weighted Close / Weighted Close Price)
3. hl2 (High-Low 2)
4. hlc3 (High-Low-Close 3)
5. kurtosis (Kurtoza)
6. log_return (Logarithmic Return)
7. mad (Mean Absolute Deviation)
8. median (Mediana)
9. midpoint (Midpoint)
10. midprice (Midprice)
11. ohlc4 (Open-High-Low-Close 4)
12. pdist (Price Distance)
13. percent_return (Percent Return)
14. quantile (Kwantyl)
15. skew (Skośność)
16. stdev (Standard Deviation)
17. variance (Wariancja)
18. zscore (Z-Score)

In [ ]:
help(ta.decay)

In [ ]:
# 1. Decay, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.decay(append=True)

In [ ]:
help(ta.hwp)

In [ ]:
# 2. Heavily Weighted Close, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.hwc(append=True)

In [ ]:
help(ta.hl2)

In [ ]:
# 3. High-Low 2, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.hl2(append=True)

In [ ]:
help(ta.hlc3)

In [ ]:
# 4. High-Low-Close 3, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.hlc3(append=True)

In [ ]:
help(ta.kurtosis)

In [ ]:
# 5. Kurtoza, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.kurtosis(append=True)

In [ ]:
help(ta.log_return)

In [ ]:
# 6. Logarithmic Return, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.log_return(append=True)

In [ ]:
help(ta.mad)

In [ ]:
# 7. Mean Absolute Deviation, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.mad(append=True)

In [ ]:
help(ta.median)

In [ ]:
# 8. Mediana, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.median(append=True)

In [ ]:
help(ta.midpoint)

In [ ]:
# 9. Midpoint, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.midpoint(append=True)

In [ ]:
help(ta.midprice)

In [ ]:
# 10. Midprice, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.midprice(append=True)

In [ ]:
help(ta.ohlc4)

In [ ]:
# 11. Open-High-Low-Close 4, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.ohlc4(append=True)

In [ ]:
help(ta.pdist)

In [ ]:
# 12. Price Distance, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.pdist(append=True)

In [ ]:
help(ta.percent_return)

In [ ]:
# 13. Percent Return, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.percent_return(append=True)

In [ ]:
help(ta.quantile)

In [ ]:
# 14. Kwantyl, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.quantile(append=True)

In [ ]:
help(ta.skew)

In [ ]:
# 15. Skewness, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.skew(append=True)

In [ ]:
help(ta.stdev)

In [ ]:
# 16. Standard Deviation, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.stdev(append=True)

In [ ]:
help(ta.variance)

In [ ]:
# 17. Variance, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.variance(append=True)

In [ ]:
help(ta.zscore)

In [ ]:
# 18. Z-Score, ustawienia standardowe
# Default Inputs:  <- parametry do modyfikacji
df.ta.zscore(append=True)

In [ ]:
df.info()